In [ ]:
#default_exp rebuild

# rebuild
> Functions needed to rebuild vision-based transforms from export

In [ ]:
#export
import io, operator, pickle, torch

In [ ]:
#export
from pathlib import Path
from numpy import ndarray
from fastcore.utils import _patched, store_attr

In [ ]:
#export
from fastinference_pytorch.transforms.vision import *
from fastinference_pytorch.transforms.data import *
from fastinference_pytorch.utils import to_device, tensor

In [ ]:
path = Path('../')

## Unpickling export

In [ ]:
#export
def load_data(path:Path=Path('.'), fn='data'):
    "Opens `pkl` file containing exported `Transform` information"
    if '.pkl' not in fn: fn += '.pkl'
    if not isinstance(path, Path): path = Path(path)
    with open(path/fn, 'rb') as handle:
        tfmd_dict = pickle.load(handle)
    return tfmd_dict

In [ ]:
#export
def load_model(path:Path=Path('.'), fn='model', cpu=True): 
    if '.pkl' not in fn: fn += '.pkl'
    if not isinstance(path, Path): path = Path(path)
    return torch.load(path/fn, map_location='cpu' if cpu else None)

In [ ]:
tfmd_dict = load_data(path)
net = load_model(path)

## `data_funcs`

`data_funcs` are designed to be quick references to tell your `Learner` how your data is going to come in. Below is an example for grabbing images, however this should be however you expect your data to be coming in:

In [ ]:
#export
def get_image(fn, mode='RGB'): return PILBase.create(fn,mode=mode)

## Pipelines

These generate our transform "pipelines" (as we're not using `fastcore`'s `Pipeline`) to pass our data through

In [ ]:
#export
def get_tfm(key, tfms): 
    "Makes a transform from `key`. Class or function must be in global memory (imported)"
    args = tfms[key]
    return globals()[key](**args)

In [ ]:
#export
def generate_pipeline(tfms, order=True) -> dict:
    "Generate `pipe` of transforms from dict and (potentially) sort them"
    pipe = []
    for key in tfms.keys():
        tfm = get_tfm(key, tfms)
        pipe.append(tfm)
    if order: pipe = sorted(pipe, key=operator.attrgetter('order'))
    return pipe

In [ ]:
#export
def make_pipelines(tfms) -> dict: 
    "Make `item` and `batch` transform pipelines"
    pipe, keys = {}, ['after_item', 'after_batch']
    for key in keys:
        pipe[key] = generate_pipeline(tfms[key], True)
    if not any(isinstance(x, ToTensor) for x in pipe['after_item']):
        pipe['after_item'].append(ToTensor(False))
    return pipe

In [ ]:
item = generate_pipeline(tfmd_dict['after_item'])
batch = generate_pipeline(tfmd_dict['after_batch'])

In [ ]:
tfmd_dict['after_batch'] = {}

In [ ]:
pipe = make_pipelines(tfmd_dict)

In [ ]:
#export
class Learner():
    """
    Similar to a `fastai` learner for inference

    Params:
      > `path`: The exact path to where your data and model is stored, relative to the `cwd`
      > `data_fn`: Filename of your pickled data
      > `model_fn`: Filename of your model
      > `data_func`: A function in which has the ability to grab your data based on some input.
                     The default grabs an image in a location and opens it with Pillow
      > `bs`: The batch size you are wanting to use per inference (this can be tweaked later)
      > `cpu`: Whether to use the CPU or GPU

    Example use:

    learn = Learner('models/data', 'models/model', data_func=image_getter, bs=4, cpu=True)
    """
    def __init__(self, path = Path('.'), data_fn='data', model_fn='model', data_func=None, bs=16, cpu=False): 
        data = load_data(path, data_fn)
        self.n_inp = data['n_inp']
        self.pipelines = make_pipelines(data)
        self.after_item = self.pipelines['after_item']
        self.after_batch = self.pipelines['after_batch']
        self.tfm_y = generate_pipeline(data['tfms'], order=False)
        self.model = load_model(path, model_fn, cpu)
        self.device = 'cpu' if cpu else 'cuda'
        store_attr(self, 'data_func,bs')
        self.decode_func = None
        
    def _make_data(self, data):
        "Passes `data` through `after_item` and `after_batch`, splitting into batches"
        self.n_batches = len(data) // self.bs + (0 if len(data)%self.bs == 0 else 1)
        batch,batches = [],[]
        for d in data:
            d = self.data_func(d)
            for tfm in self.after_item: d = tfm(d)
            batch.append(d)
            if len(batch) == self.bs or (len(batches) == self.n_batches - 1 and len(batch) == len(data)):
                batch = torch.stack(batch, dim=0)
                batch = to_device(batch, self.device)
                for tfm in self.after_batch: 
                    batch = tfm(batch)
                batches.append(batch)
                batch = []
        return batches
    
    def _decode_inputs(self, inps, outs):
        "Decodes images through `after_batch`"
        for tfm in self.after_batch[::-1]:
            if hasattr(tfm, 'can_decode'):
                inps = to_device(tfm(inps, decode=True))
        outs.insert(len(outs), inps)
        return outs
    
    def get_preds(self, data, raw_outs=False, decode_func=None, with_input=False):
        """
        Gather predictions on `data` with possible decoding. 
        
        Params:
          > `data`: Incoming data formatted to what `self.data_func`is expecting
          > `raw_outs`: Whether to return the raw outputs
          > `decode_func`: A function to use for decoding potential outputs.
                           While the default is `None`, see `decode_cel` for an example
          > `with_input`: Whether to return a decoded input up to what the model was passed
        """
        inps, outs, dec_out, raw = [],[],[],[]
        batches = self._make_data(data)
        if self.n_inp > 1:
            [inps.append([]) for _ in range(n_inp)]
        with torch.no_grad():
            for batch in batches:
                if with_input:
                    if self.n_inp > 1:
                        for i in range(self.n_inp):
                            inps[i].append(batch[i].cpu())
                        else:
                            inps.append(batch[0].cpu())
                self.model.eval()
                out = self.model(batch[:self.n_inp])
            raw.append(out)
            if self.decode_func is not None: dec_out.append(decode_func(out))
        raw = torch.cat(raw, dim=0).cpu().numpy()
        if self.decode_func is not None: dec_out = torch.cat(dec_out, dim=0)
        if self.decode_func is None or raw_outs: outs.insert(0, raw)
        else: outs.insert(0, raw)
        
        if with_input: outs = self._decode_inputs(inps, outs)
        return outs

In [ ]:
learn = Learner(path='../', data_func=get_image)

In [ ]:
data = [path/'forest.jpg', path/'forest-Copy1.jpg']

In [ ]:
%%time
_ = learn._make_data(data)

CPU times: user 101 ms, sys: 15.4 ms, total: 117 ms
Wall time: 104 ms


In [ ]:
%%time
preds = learn.get_preds(data)

CPU times: user 103 ms, sys: 20.8 ms, total: 124 ms
Wall time: 111 ms


In [ ]:
preds

[array([[ 5.075976  , -1.2860683 ,  0.28139827,  1.3314227 ,  0.24555245,
         -1.267573  ,  0.839015  , -6.593617  , -2.1617684 , -1.9049999 ,
         -2.9269905 ,  5.585224  ,  3.2263792 , -1.6155014 , -0.6794665 ,
          3.3905056 , -2.9616075 , -1.4418863 ,  0.9092367 , -1.1158583 ,
          1.2579541 ,  4.975295  , -0.7700971 , -0.5027274 ,  1.9125727 ,
         -0.46821177, -3.2195568 ,  1.3048875 ,  2.3925881 , -2.7260544 ,
         -6.7417536 ,  4.4691014 ,  3.138262  , -4.310676  , -1.2019831 ,
         -0.27300605,  4.1942105 ]], dtype=float32)]

In [ ]:
data[0].shape

torch.Size([2, 3, 460, 460])

In [ ]:
out.shape

torch.Size([2, 37])